In [5]:
import pandas as pd 
import numpy as np 

title = ['source','target','time']
data_all = pd.read_csv('sx-mathoverflow.txt',header=None,sep=' ',names=title)
# a2q
data1 = pd.read_csv('sx-mathoverflow-a2q.txt',header=None,sep=' ',names=title)
# c2a
data2 = pd.read_csv('sx-mathoverflow-c2a.txt',header=None,sep=' ',names=title)
# c2q
data3 = pd.read_csv('sx-mathoverflow-c2q.txt',header=None,sep=' ',names=title)

In [6]:
data1['label'] = 'a2q'
data2['label'] = 'c2a'
data3['label'] = 'c2q'

data0 = pd.concat([data1,data2,data3])
data = pd.merge(data_all,data0,on=title)
data.head()

,source,target,time,label
0,1,4,1254192988,a2q
1,3,4,1254194656,a2q
2,1,2,1254202612,a2q
3,25,1,1254232804,a2q
4,14,16,1254263166,a2q


In [17]:
data.drop_duplicates()
data[['source','target','label']].to_csv('math_all.txt',sep='\t',index=None,header=None)

In [19]:
print(data_all.size/3,data1.size/3,data2.size/3,data3.size/2,data0.size/3,data.size/4)

506550.0 143441.33333333334 260440.0 407278.0 675400.0 506614.0


In [22]:
#print(data[data.duplicated()].count())
print(data.size/4)

506614.0


In [24]:
dd1 = data.set_index('target')['source'].to_dict()
dd2 = data.set_index('source')['target'].to_dict()

k1 = list(dd1.keys())
k2 = list(dd2.keys())

print(len(k1),len(k2))

21980 19774


In [25]:
en = list(set(k1).union(set(k2)))
print(len(en))

entities = pd.DataFrame(en)
entities['counter'] = range(len(entities))
print(entities.head())

24818
   0  counter
0  1        0
1  2        1
2  3        2
3  4        3
4  7        4


In [26]:
entities.to_csv('entity2id.txt',header=None,sep='\t',index=None)

In [27]:
entities.columns = ['entity','counter']
d = entities.set_index('entity')['counter'].to_dict()

r = ['a2q','c2a','c2q']
rr = pd.DataFrame(r)
rr['counter'] = range(len(rr))
rr.columns = ['label','counter']
print(rr.head())

  label  counter
0   a2q        0
1   c2a        1
2   c2q        2


In [28]:
rr.to_csv('relation2id.txt',header=None,sep='\t',index=None)

In [29]:
r0 = rr.set_index('label')['counter'].to_dict()
def fun(x,d):
    return d[x]
data['t'] = data['target'].apply(lambda x: fun(x,d))
data['s'] = data['source'].apply(lambda x: fun(x,d))
data['r'] = data['label'].apply(lambda x: fun(x,r0))
print(data.head())

   source  target        time label  t   s  r
0       1       4  1254192988   a2q  3   0  0
1       3       4  1254194656   a2q  3   2  0
2       1       2  1254202612   a2q  1   0  0
3      25       1  1254232804   a2q  0  13  0
4      14      16  1254263166   a2q  8   7  0


In [30]:
f_data = data[['s','t','r']]
f_data.to_csv('math_all.txt',header=None,sep='\t',index=None)

In [31]:
rand_train_all = f_data.sample(frac=0.8)
rand_test = f_data[~f_data.index.isin(rand_train_all.index)]

rand_train_all.to_csv('train_all.txt',header=None,sep='\t',index=None)
rand_test.to_csv('test2id.txt',header=None,sep='\t',index=None)

rand_train = rand_train_all.sample(frac=0.75)
rand_valid = rand_train_all[~rand_train_all.index.isin(rand_train.index)]

rand_train.to_csv('train2id.txt',header=None,sep='\t',index=None)
rand_valid.to_csv('valid2id.txt',header=None,sep='\t',index=None)